In [84]:
from xml.dom import minidom
import regex

In [85]:
# structure to expect:
#	onixmessage
#		product
#			publishingdetail
#				salesrights
#					territory
#						countriesincluded/‘worldwide with specified exclusions’/worldwide/etc.

# onix message- mandatory

# product- can expect multiple per file

# publishingdetail- "It is mandatory in any <Product> record unless the <NotificationType> in Group P.1 indicates that the record is an update notice which carries only those blocks in which changes have occurred."

# salesrights- "allows rights to be specified as exclusive or non-exclusive or not-for-sale in any combination of countries or regions. It is also possible to specify rights as ‘worldwide’ or ‘worldwide with specified exclusions’ if this enables them to be stated more concisely."

# territory- "non-repeating. If omitted, the text is intended for use wherever the product may be sold"

# tool functions

In [86]:

######### some styling functions to make error reporting more legible. ########

def boldQuote(content):
    out =  '\033[91m'+'\033[1m'+'"'+ content +'"'+'\033[0m'
    return out

def notify(content):
    out =  '\033[95m'+ content +'\033[0m'
    return out

def error(content):
    out =  '\033[91m'+'\033[1m'+ content +'\033[0m'
    return out

######### clean tag name for checks to allow for caps, whitespace, punctuation etc. ########
# A good way to extend this out would be to check for 
# eg. common spelling mistakes.

def cleanText(string):
    
    #clean out whitespace
    out = string.replace(" ", "")
    
    #revert to lowercase
    out = out.lower()
    
    #clean all punctuation and non latin characters (excluding numbers)
    cleanout = regex.sub('[^a-zA-Z0-9]', u'', out)
    
    if(cleanout != out):
        print ('XML tag',boldQuote(out),'translated to',boldQuote(cleanout))

    return cleanout


def searchLayer( layer, searchTerm, prevLayer ):
    
    # see whether we have multiple tags and report back.
    if (len(layer) > len(prevLayer)):
        multTagName = layer[0].tagName
        print(notify('number of '+ multTagName + ' tags in ' + currentFile + ' is greater than 1'))
        permittedNumberOfTags = len(layer)
    
    # childNodes can't be found if the div isn't an element, so use except to skip other types
    # pull out all tags which have a name matching our searchterm, and collect them in an array
    # which we will push into currentTag.
    
    try:
        
        out = []
        
        for i in range(len(layer)):
            items = layer[i].childNodes

            for item in items:
                try:
                    elementTag = cleanText(item.tagName)
                    if (elementTag == searchTerm):
                        out.append(item)
                except:
                    pass

            if (len(out) == 0):
                print(error('no ' + searchTerm + ' tag found in ' + currentFile))
                
        return out
    
    except:
        pass
    
def getBookNames(currentTitleTag, titleTagTypes):
    bookNamesInFile = []
    for i in range(len(currentTitleTag)):
        bookName = ""
        for node in currentTitleTag[i].childNodes:
            try:
                titleTag = node.childNodes[0]
                title = titleTag.nodeValue
                
                if (cleanText(node.tagName) in titleTagTypes):
                    if(len(bookName) > 0):
                        bookName += " "
                    bookName += title
            except:
                pass
        bookNamesInFile.append(bookName)
    return bookNamesInFile

def getCountryLists(currentCountryTag):
    countryListsInFile = []
    for i in range(len(currentCountryTag)):
        for node in currentCountryTag[i].childNodes:
            try:
                #we only want nodes which have a childnode, as countrylists are stored
                #within the childnode. I'm going to assume they aren't going to be 
                #badly written and stored in eg. tagnames for the scope of this project.

                countriesTag = node.childNodes[0].nodeValue                
                countriesArray = countriesTag.split()
                
                checkDeprecatedCountryCodes(countriesArray)
                
                # IMPORTANT NOTE: ‘worldwide with specified exclusions’ is a permitted format for territory. 
                # This checks for that case.
                countriesArray = checkExcluded(countriesArray)

            except:
                pass
        
        countriesArray = checkWorld(countriesArray, allCountries)
            
        countryListsInFile.append(countriesArray)
    return countryListsInFile
    

# test functions

In [87]:
def checkDeprecatedCountryCodes(countriesArray):
    for countryCode in countriesArray:
        if (countryCode == "YU" or countryCode =="CS" or countryCode == 'AN'):
            print(error('WARNING: country code "' + countryCode + '" in territories for "' + bookName + '" in '+ currentFile +' is deprecated'))

def checkWorld(countriesArray, allCountries):
    if (cleanText(countriesArray[0]) == 'world'):
        countriesArray = allCountries
    return countriesArray

# countries list gathered from here: https://www.editeur.org/files/ONIX%20for%20books%20-%20code%20lists/ONIX_BookProduct_Codelists_Issue_49.html
# list 91. with CS - Serbia & Montenegro, AN- Netherlands Antilles and 
# YU - Yugoslavia excluded. 
# Warnings are given below for deprecated CS, AN, and YU countrycodes.

def checkExcluded(countriesArray):
    if (cleanText(node.tagName) == 'countriesexcluded' or cleanText(node.tagName) == 'x451'):
        subtractedCountriesArray = allCountries
        for i in range(len(countriesArray)):
            if( countriesArray[i] in subtractedCountriesArray ):
                subtractedCountriesArray.remove(countriesArray[i])                        
        countriesArray = subtractedCountriesArray
    return countriesArray

def checkMissingTerritoryTag(currentCountryTag, countryLists, allCountries):
    # "If <territory tag> is omitted, the text is intended for use 
    # wherever the product may be sold" - Onix Format specification
    if (len(currentCountryTag) == 0):
        print(notify('no territory tag so all countries included'))
        countriesArray = allCountries
        countryLists.append(countriesArray)
    return countryLists
    
def checkTitleCountriesLengths(bookNames, countryLists):
    if(len(bookNames) != len(countryLists)):
        print(error('WARNING: different number of book names found in file than territory codes.'))
    

# process the data

In [88]:
docs = ['1','2','3','4']
# docs=["test2"]
# docs=["test"]

# strictly speaking, we should probably search for information on permitted countries of sale
# within all tags within the salesrights div, as this is the extent of the purpose of
# the salesrights div, (and I think it is likely for salesrights information to be misplaced 
# outside of a territory tag) but 'territory' is specified in the ONIX documentation as 
# non-repeating, and, if ommitted, 'the text is intended for use whereever it may be sold', 
# so I will default to that understanding.

# so, we want to 'dig down' through all of these tagNames to find our countrylist

# NOTE: short tags for all of the following searchtags are lowercase, so are searched 
# for with .lower().

tagNamesCountries = ['product', 'publishingdetail', 'salesrights', 'territory']
tagNamesTitle = ['product', 'descriptivedetail', 'titledetail', 'titleelement']

# tags which could contain title information for the book in question.
# partnumbers and annual numbers and subtitles included/appended to 
# improve the chances of titles being unique.

titleTagTypes = ['titletext', 'b203', 'titlewithoutprefix', 'b031', 'yearofannual', 'b020', 'partnumber', 'x410', 'subtitle', 'b029']

# countries list gathered from here: https://www.editeur.org/files/ONIX%20for%20books%20-%20code%20lists/ONIX_BookProduct_Codelists_Issue_49.html
# list 91. with CS - Serbia & Montenegro, AN- Netherlands Antilles and 
# YU - Yugoslavia excluded. 

allCountries = ['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AW', 'AX', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BL', 'BM', 'BN', 'BO', 'BQ', 'BR', 'BS', 'BT', 'BV', 'BW', 'BY', 'BZ', 'CA', 'CC', 'CD', 'CF', 'CG', 'CH', 'CI', 'CK', 'CL', 'CM', 'CN', 'CO', 'CR', 'CU', 'CV', 'CW', 'CX', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'EH', 'ER', 'ES', 'ET', 'FI', 'FJ', 'FK', 'FM', 'FO', 'FR', 'GA', 'GB', 'GD', 'GE', 'GF', 'GG', 'GH', 'GI', 'GL', 'GM', 'GN', 'GP', 'GQ', 'GR', 'GS', 'GT', 'GU', 'GW', 'GY', 'HK', 'HM', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IM', 'IN', 'IO', 'IQ', 'IR', 'IS', 'IT', 'JE', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KP', 'KR', 'KW', 'KY', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MF', 'MG', 'MH', 'MK', 'ML', 'MM', 'MN', 'MO', 'MP', 'MQ', 'MR', 'MS', 'MT', 'MU', 'MV', 'MW', 'MX', 'MY', 'MZ', 'NA', 'NC', 'NE', 'NF', 'NG', 'NI', 'NL', 'NO', 'NP', 'NR', 'NU', 'NZ', 'OM', 'PA', 'PE', 'PF', 'PG', 'PH', 'PK', 'PL', 'PM', 'PN', 'PR', 'PS', 'PT', 'PW', 'PY', 'QA', 'RE', 'RO', 'RS', 'RU', 'RW', 'SA', 'SB', 'SC', 'SD', 'SE', 'SG', 'SH', 'SI', 'SJ', 'SK', 'SL', 'SM', 'SN', 'SO', 'SR', 'SS', 'ST', 'SV', 'SX', 'SY', 'SZ', 'TC', 'TD', 'TF', 'TG', 'TH', 'TJ', 'TK', 'TL', 'TM', 'TN', 'TO', 'TR', 'TT', 'TV', 'TW', 'TZ', 'UA', 'UG', 'UM', 'US', 'UY', 'UZ', 'VA', 'VC', 'VE', 'VG', 'VI', 'VN', 'VU', 'WF', 'WS', 'YE', 'YT', 'ZA', 'ZM', 'ZW']

allBooks = []
bookNames = []
countryLists = []


for doc in docs:
    
    currentFile = doc + '.xml'
    mydoc = minidom.parse( currentFile )
    
    try:
        onixWrapper = mydoc.getElementsByTagName("ONIXmessage")
    except IndexError:
        print('no ONIX Wrapper in ' + currentFile)
        pass
    
    currentCountryTag = onixWrapper
    currentTitleTag = onixWrapper
    prevTag = currentCountryTag
    
    # use the searchLayer function to check the file for tags of the right tagName,
    # collect them in an array and then search those tags for the next correct tagName
    # to find the nesting where the countries info is stored.
    # includes error reporting and checks
    
    for layer in tagNamesCountries:
        newTag = searchLayer(currentCountryTag, layer, prevTag)
        prevTag = currentCountryTag
        currentCountryTag = newTag
        
    for layer in tagNamesTitle:
        currentTitleTag = searchLayer(currentTitleTag, layer, prevTag)
        
    # get the titles of the books in the file
    
    bookNamesInFile = getBookNames(currentTitleTag, titleTagTypes)
    bookNames.extend(bookNamesInFile)
    
    # get the permitted countries of sale for each book
    
    countryListsInFile = getCountryLists(currentCountryTag)
    countryLists.extend(countryListsInFile)
    
    countryLists = checkMissingTerritoryTag(currentCountryTag, countryLists, allCountries)
    
    checkTitleCountriesLengths(bookNames, countryLists)
    
# now that we've finally found all the book names and territory lists in all the files, 
# put them in a dictionary so they're nice and straightforward to put in a database.
# this could be easily extended to include more info.
    
for i in range(len(bookNames)):    
    book = {}
    book['title'] = bookNames[i]
    book['countries'] = countryLists[i]
    allBooks.append(book)

# print(allBooks)

# databasing


In [89]:
import mysql
import sqlalchemy as sqla
from sqlalchemy.sql import text
metadata = sqla.MetaData()


host = 'sql2.freesqldatabase.com'
database_name = 'sql2395730'
user = 'sql2395730'
password = 'fL7!xU9%'
port = '3306'

engine = sqla.create_engine(
    'mysql+mysqlconnector://'+user+':'+password+'@'+host+':'+port+'/'+database_name
)

In [90]:
#SQL queries to build the tables

# #World record longest title of a book consists of 26,021 characters, text can have 65,535 chars.
createBookTable  = text("""CREATE TABLE Books ( 
     id int primary key not null auto_increment,
     Title text
    )""")

# #Longest permitted country code by ONIX standards is 2,  30 is plenty
createCountryTable  = text("""CREATE TABLE Countries (
   id int primary key not null auto_increment,
   Country varchar(30)
    )""")

createBookRightsTable  = text("""CREATE TABLE BookRights (
   CountryID int references Countries(ID),
   BookID int references Books(id),
   constraint pk_Books primary key (CountryID, BookID)
   )""")

#Build the tables
# conn = engine.connect()
# conn.execute(createBookTable)
# conn.execute(createCountryTable)
# conn.execute(createBookRightsTable)
# conn.close()

In [91]:
#Fill the Books and Countries Tables

print(engine.table_names())

def insertIntoDb( data, table, column ):
    try:
        insert_query = text("""INSERT INTO """ + table + """ 
                       ("""+column+""") 
                       VALUES(:title)""")

        conn.execute(insert_query, data)

    except (Exception) as error :
        print(data)
        print("Failed to insert record into table:", error)
        raise error

conn = engine.connect()

# duplicates if more books are added- would be good as an extension to check if the book already exists
# in the database and return a warning. 

# for book in bookNames:
#     bookItem = {}
#     bookItem['title'] = book
#     record_to_insert = ( bookItem )
#     insertIntoDb( record_to_insert, 'Books', 'Title' )
    
# only run once because we have a set list of possible country codes

# for country in allCountries:
#     countryItem = {}
#     countryItem['title'] = country
#     record_to_insert = ( countryItem )
#     insertIntoDb( record_to_insert, 'Countries', 'Country')
    
conn.close()

['BookRights', 'Books', 'Countries']


In [94]:
def crossReference( data, table, column ):
    try:
        insert_query = text("""INSERT INTO """ + table + """ 
                       ("""+column+""") 
                       VALUES(:title)""")

        conn.execute(insert_query, data)

    except (Exception) as error :
        print(data)
        print("Failed to insert record into table:", error)
        raise error
    
conn = engine.connect()

for book in allBooks:
    bookResult = conn.execute("SELECT id FROM Books WHERE Title = '"+ book['title'] +"'")
    bookID = bookResult.fetchone()[0]
    
    for country in book['countries']:

        countryResult = conn.execute("SELECT id FROM Countries WHERE Country = '"+ country +"'")
        
#         print('inserting',book['title'], bookID, country, '...')
        
        #if the countryID exists (is not deprecated), insert the data relation, otherwise pass.
        countryIDItem = countryResult.fetchone()
        if countryIDItem:
            countryID = countryIDItem[0]

            print('inserting',book['title'], bookID, country, countryID,'...')

            insert_query = text("""INSERT into BookRights(BookID, CountryID) 
                            VALUES("""+ str(bookID) + """,""" + str(countryID) + """)""" )

            conn.execute(insert_query)
        
        else:
            print(error('skipping '+ str(country) +' as ID is not found in permitted countries table'))

print('finished, closing connection')
conn.close()

inserting Demo The Demo Book 33 AD 254 ...
inserting Demo The Demo Book 33 AE 255 ...
inserting Demo The Demo Book 33 AF 256 ...
inserting Demo The Demo Book 33 AG 257 ...
inserting Demo The Demo Book 33 AI 258 ...
inserting Demo The Demo Book 33 AL 259 ...
inserting Demo The Demo Book 33 AM 260 ...
inserting Demo The Demo Book 33 AO 261 ...
inserting Demo The Demo Book 33 AQ 262 ...
inserting Demo The Demo Book 33 AR 263 ...
inserting Demo The Demo Book 33 AS 264 ...
inserting Demo The Demo Book 33 AT 265 ...
inserting Demo The Demo Book 33 AU 266 ...
inserting Demo The Demo Book 33 AW 267 ...
inserting Demo The Demo Book 33 AX 268 ...
inserting Demo The Demo Book 33 AZ 269 ...
inserting Demo The Demo Book 33 BA 270 ...
inserting Demo The Demo Book 33 BB 271 ...
inserting Demo The Demo Book 33 BD 272 ...
inserting Demo The Demo Book 33 BE 273 ...
inserting Demo The Demo Book 33 BF 274 ...
inserting Demo The Demo Book 33 BG 275 ...
inserting Demo The Demo Book 33 BH 276 ...
inserting D

inserting Demo The Demo Book 33 RW 445 ...
inserting Demo The Demo Book 33 SA 446 ...
inserting Demo The Demo Book 33 SB 447 ...
inserting Demo The Demo Book 33 SC 448 ...
inserting Demo The Demo Book 33 SD 449 ...
inserting Demo The Demo Book 33 SE 450 ...
inserting Demo The Demo Book 33 SG 451 ...
inserting Demo The Demo Book 33 SH 452 ...
inserting Demo The Demo Book 33 SI 453 ...
inserting Demo The Demo Book 33 SJ 454 ...
inserting Demo The Demo Book 33 SK 455 ...
inserting Demo The Demo Book 33 SL 456 ...
inserting Demo The Demo Book 33 SM 457 ...
inserting Demo The Demo Book 33 SN 458 ...
inserting Demo The Demo Book 33 SO 459 ...
inserting Demo The Demo Book 33 SR 460 ...
inserting Demo The Demo Book 33 SS 461 ...
inserting Demo The Demo Book 33 ST 462 ...
inserting Demo The Demo Book 33 SV 463 ...
inserting Demo The Demo Book 33 SX 464 ...
inserting Demo The Demo Book 33 SY 465 ...
inserting Demo The Demo Book 33 SZ 466 ...
inserting Demo The Demo Book 33 TC 467 ...
inserting D

inserting Demo The Demo 4 Book 36 KG 369 ...
inserting Demo The Demo 4 Book 36 LA 379 ...
inserting Demo The Demo 4 Book 36 LV 388 ...
inserting Demo The Demo 4 Book 36 LB 380 ...
inserting Demo The Demo 4 Book 36 LS 385 ...
inserting Demo The Demo 4 Book 36 LR 384 ...
inserting Demo The Demo 4 Book 36 LY 389 ...
inserting Demo The Demo 4 Book 36 LI 382 ...
inserting Demo The Demo 4 Book 36 LT 386 ...
inserting Demo The Demo 4 Book 36 LU 387 ...
inserting Demo The Demo 4 Book 36 MK 397 ...
inserting Demo The Demo 4 Book 36 MG 395 ...
inserting Demo The Demo 4 Book 36 MW 409 ...
inserting Demo The Demo 4 Book 36 MY 411 ...
inserting Demo The Demo 4 Book 36 MV 408 ...
inserting Demo The Demo 4 Book 36 ML 398 ...
inserting Demo The Demo 4 Book 36 MT 406 ...
inserting Demo The Demo 4 Book 36 MH 396 ...
inserting Demo The Demo 4 Book 36 MQ 403 ...
inserting Demo The Demo 4 Book 36 MR 404 ...
inserting Demo The Demo 4 Book 36 MU 407 ...
inserting Demo The Demo 4 Book 36 YT 499 ...
inserting 

In [99]:
#Now we've put the data in the database, we can query it! 
#Here's an example where we look at the countries an alphabetical order alongside the 
#books they have allocated to them.

conn = engine.connect()
bookResult = conn.execute(text("""
    SELECT * FROM Books B INNER JOIN BookRights BR ON BR.BookID = B.id INNER JOIN Countries C On BR.CountryID = C.id
    order by C.id;
    """))

items = bookResult.fetchall()
for item in items:
    print(item[1],",",item[5])
conn.close()

Demo The Demo Book , AD
Demo The Demo 4 Book , AD
Demo The Demo Book , AE
Demo The Demo 4 Book , AE
Demo The Demo Book , AF
Demo The Demo Book , AG
Demo The Demo 4 Book , AG
Demo The Demo Book , AI
Demo The Demo 4 Book , AI
Demo The Demo Book , AL
Demo The Demo 4 Book , AL
Demo The Demo Book , AM
Demo The Demo 4 Book , AM
Demo The Demo Book , AO
Demo The Demo 4 Book , AO
Demo The Demo Book , AQ
Demo The Demo 4 Book , AQ
Demo The Demo Book , AR
Demo The Demo 4 Book , AR
Demo The Demo Book , AS
Demo The Demo 4 Book , AS
Demo The Demo Book , AT
Demo The Demo 4 Book , AT
Demo The Demo Book , AU
Demo The Demo 4 Book , AU
Demo The Demo Book , AW
Demo The Demo 4 Book , AW
Demo The Demo Book , AX
Demo The Demo 4 Book , AX
Demo The Demo Book , AZ
Demo The Demo 4 Book , AZ
Demo The Demo Book , BA
Demo The Demo 4 Book , BA
Demo The Demo Book , BB
Demo The Demo 4 Book , BB
Demo The Demo Book , BD
Demo The Demo 4 Book , BD
Demo The Demo Book , BE
Demo The Demo 4 Book , BE
Demo The Demo Book , BF
De

Demo The Demo 4 Book , UM
Demo The Demo Book , US
Demo The Demo 2 Book , US
Demo The Demo 4 Book , US
Demo The Demo Book , UY
Demo The Demo 4 Book , UY
Demo The Demo Book , UZ
Demo The Demo 4 Book , UZ
Demo The Demo Book , VA
Demo The Demo 4 Book , VA
Demo The Demo Book , VC
Demo The Demo 4 Book , VC
Demo The Demo Book , VE
Demo The Demo 4 Book , VE
Demo The Demo Book , VG
Demo The Demo 4 Book , VG
Demo The Demo Book , VI
Demo The Demo 4 Book , VI
Demo The Demo Book , VN
Demo The Demo 4 Book , VN
Demo The Demo Book , VU
Demo The Demo 4 Book , VU
Demo The Demo Book , WF
Demo The Demo 4 Book , WF
Demo The Demo Book , WS
Demo The Demo 4 Book , WS
Demo The Demo Book , YE
Demo The Demo 4 Book , YE
Demo The Demo Book , YT
Demo The Demo 4 Book , YT
Demo The Demo Book , ZA
Demo The Demo 4 Book , ZA
Demo The Demo Book , ZM
Demo The Demo 4 Book , ZM
Demo The Demo Book , ZW
Demo The Demo 4 Book , ZW


# tests thoughts

###### ONIX typically contains multiple products

##### is there a wrapping ONIXMessage declaration?
##### is there a header tag?

##### is there a product tag? case - no product tag - there can (and often is) multiple product tags

##### is there a territory tag? (should be within product tag) case- no territory tag 

###### case- territory tag found outside product tag

##### what are the permitted country codes within the territory tag? (look to be typically 2 letters but can be 'WORLD')

##### case- unpermitted country code(s)

##### case- no country codes

##### multiple 'product', 'publishingdetail', 'salesrights', 'territory' tags

##### one or multiple tags missing in 'product', 'publishingdetail', 'salesrights', 'territory' chain